In [ ]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

# Прочтение таблиц

In [ ]:
df = pd.read_excel('/content/Lychelle.xlsx')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,77765,0.093740,TGTGCCAGCAGTTCGGTGGGACTATACTACGAGCAGTACTTC,CASSSVGLYYEQYF,TRBV12-4,TRBJ2-7
1,1,18548,0.022358,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5
2,2,8644,0.010420,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1
3,3,7160,0.008631,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6
4,4,6128,0.007387,TGCGCCAGCAGCCAAGATCTAGGGGGGCCCACAGATACGCAGTATTTT,CASSQDLGGPTDTQYF,TRBV4-3,TRBJ2-3


In [ ]:
database = pd.read_excel('/content/vdjdb_slim.xlsx')

In [ ]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [ ]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,77765,0.093740,TGTGCCAGCAGTTCGGTGGGACTATACTACGAGCAGTACTTC,CASSSVGLYYEQYF,TRBV12-4,TRBJ2-7
1,1,18548,0.022358,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5
2,2,8644,0.010420,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1
3,3,7160,0.008631,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6
4,4,6128,0.007387,TGCGCCAGCAGCCAAGATCTAGGGGGGCCCACAGATACGCAGTATTTT,CASSQDLGGPTDTQYF,TRBV4-3,TRBJ2-3
...,...,...,...,...,...,...,...
9995,11357,4,0.000005,TGCGCCAGCAGCTTGGGATACGGCGGGAAGACCCAGTACTTC,CASSLGYGGKTQYF,TRBV5-1,TRBJ2-5
9996,11351,4,0.000005,TGCGCCAGCAGCTTGGGCAGGACGTCGGAGACCCAGTACTTC,CASSLGRTSETQYF,TRBV5-1,TRBJ2-5
9997,9298,4,0.000005,TGCGCCAGCAGCTTGGGCGAAGAGACAGGGAACACCGGGGAGCTGT...,CASSLGEETGNTGELFF,TRBV5-1,TRBJ2-2
9998,12531,4,0.000005,TGCGCCAGCAGCTTGGGCTACGAGCAGTACTTC,CASSLGYEQYF,TRBV5-1,TRBJ2-7


# Функция кластеризации

In [ ]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [ ]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,77765,0.093740,TGTGCCAGCAGTTCGGTGGGACTATACTACGAGCAGTACTTC,CASSSVGLYYEQYF,TRBV12-4,TRBJ2-7,1
1,1,18548,0.022358,TGTGCCAGCAGTTTATACGGGGACCAAGAGACCCAGTACTTC,CASSLYGDQETQYF,TRBV27,TRBJ2-5,2
2,2,8644,0.010420,TGTGCCAGCAGTTTTCGGACAGGGCCCCCGCCCTACAATGAGCAGT...,CASSFRTGPPPYNEQFF,TRBV6-2,TRBJ2-1,3
3,3,7160,0.008631,TGTGCCAGCAGATCTGGGGCCAACGTCCTGACTTTC,CASRSGANVLTF,TRBV4-2,TRBJ2-6,4
4,4,6128,0.007387,TGCGCCAGCAGCCAAGATCTAGGGGGGCCCACAGATACGCAGTATTTT,CASSQDLGGPTDTQYF,TRBV4-3,TRBJ2-3,5
...,...,...,...,...,...,...,...,...
9995,11357,4,0.000005,TGCGCCAGCAGCTTGGGATACGGCGGGAAGACCCAGTACTTC,CASSLGYGGKTQYF,TRBV5-1,TRBJ2-5,8825
9996,11351,4,0.000005,TGCGCCAGCAGCTTGGGCAGGACGTCGGAGACCCAGTACTTC,CASSLGRTSETQYF,TRBV5-1,TRBJ2-5,8826
9997,9298,4,0.000005,TGCGCCAGCAGCTTGGGCGAAGAGACAGGGAACACCGGGGAGCTGT...,CASSLGEETGNTGELFF,TRBV5-1,TRBJ2-2,8827
9998,12531,4,0.000005,TGCGCCAGCAGCTTGGGCTACGAGCAGTACTTC,CASSLGYEQYF,TRBV5-1,TRBJ2-7,1711


# Подсчёт Total Read Proportion

In [ ]:
# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})


# Сортировка по Sequence Count

In [ ]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
5,6,0.006525,54
227,228,0.000580,41
242,243,0.000755,30
67,68,0.001639,28
355,356,0.000399,28
...,...,...,...
3136,3140,0.000012,1
3135,3139,0.000012,1
3134,3138,0.000012,1
3133,3137,0.000012,1


# Сортировка по Total Read Proportion

In [ ]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
3,4,0.008636,2
4,5,0.007828,20
5,6,0.006525,54
23,24,0.004191,3
9,10,0.004182,4
...,...,...,...
8612,8616,0.000010,2
8065,8069,0.000010,2
8083,8087,0.000010,2
8328,8332,0.000010,2


# Выбор кластера

In [ ]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [ ]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,42,CASSPGQNEQFF,0.001057,TRBV9,TRBJ2-1,TRBV18*01,TRBJ2-1*01,MHCII,HomoSapiens,InfluenzaA,HA
1,111,CASSLGGNTGELFF,0.000331,TRBV12-4,TRBJ2-2,TRBV27*01,TRBJ2-2*01,MHCI,HomoSapiens,InfluenzaA,M
2,111,CASSLGGNTGELFF,0.000295,TRBV12-4,TRBJ2-2,TRBV27*01,TRBJ2-2*01,MHCI,HomoSapiens,InfluenzaA,M
3,58,CASSYGGDGYTF,0.000219,TRBV6-5,TRBJ1-2,TRBV6-5*01,TRBJ1-2*01,MHCI,HomoSapiens,CMV,IE1
4,227,CSNQPQHF,0.000149,TRBV15,TRBJ1-5,TRBV20-1*01,TRBJ1-5*01,MHCI,HomoSapiens,HomoSapiens,MLANA
...,...,...,...,...,...,...,...,...,...,...,...
393,1711,CASSLGYEQYF,0.000005,TRBV5-1,TRBJ2-7,"TRBV7-8*01,TRBV7-9*01,TRBV5-1*01",TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1
394,1711,CASSLGYEQYF,0.000005,TRBV5-1,TRBJ2-7,TRBV7-8*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,EBNA4
395,1711,CASSLGYEQYF,0.000005,TRBV5-1,TRBJ2-7,"TRBV7-8*01,TRBV27*01",TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
396,1711,CASSLGYEQYF,0.000005,TRBV5-1,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,HomoSapiens,MLANA


In [ ]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
3,58,CASSYGGDGYTF,0.000219,TRBV6-5,TRBJ1-2,TRBV6-5*01,TRBJ1-2*01,MHCI,HomoSapiens,CMV,IE1
12,279,CASSLGPNTEAFF,0.000082,TRBV12-4,TRBJ1-1,"TRBV12-3*01,TRBV7-2*01",TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
13,279,CASSLGPNTEAFF,0.000082,TRBV12-4,TRBJ1-1,TRBV12-3*01,TRBJ1-1*01,MHCI,HomoSapiens,EBV,EBNA4
24,501,CASSLGQGDGYTF,0.000061,TRBV7-9,TRBJ1-2,TRBV7-9*01,TRBJ1-2*01,MHCI,HomoSapiens,HomoSapiens,MLANA
32,624,CASSDSGTDTQYF,0.000052,TRBV6-4,TRBJ2-3,TRBV6-4*01,TRBJ2-3*01,MHCI,HomoSapiens,YFV,NS4B
39,989,CASSLGTGELFF,0.000035,TRBV12-4,TRBJ2-2,TRBV12-4*01,TRBJ2-2*01,MHCI,HomoSapiens,CMV,IE1
52,116,CASSDSSGGNEQFF,0.000028,TRBV6-4,TRBJ2-1,TRBV6-4*01,TRBJ2-1*01,MHCI,HomoSapiens,CMV,IE1
68,1185,CASSLEETQYF,0.000017,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
79,1185,CASSLGETQYF,0.000016,TRBV11-2,TRBJ2-5,TRBV11-2*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,pp65
84,2519,CASSELAGGPDTQYF,0.000016,TRBV6-1,TRBJ2-3,TRBV6-1*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1


In [ ]:
full_vj['Cluster'].value_counts()

,count
Cluster,
1185,8


# Обработка для одного выбранного кластера

In [ ]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 1185]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
1248,1248,25,0.000030,TGTGCCAGCAGTTTAAGGGAAGAGACCCAGTACTTC,CASSLREETQYF,TRBV25-1,TRBJ2-5,1185
2384,2523,14,0.000017,TGCGCCAGCAGCTTGGAAGAGACCCAGTACTTC,CASSLEETQYF,TRBV5-1,TRBJ2-5,1185
2450,2524,14,0.000017,TGTGCCAGCAGCTTAGAGACCCAGTACTTC,CASSLETQYF,TRBV11-2,TRBJ2-5,1185
2673,2756,13,0.000016,TGTGCCAGCAGCTTAGGAGAGACCCAGTACTTC,CASSLGETQYF,TRBV11-2,TRBJ2-5,1185
2721,2753,13,0.000016,TGTGCCAGCAGTTTAGCAGAGACCCAGTACTTC,CASSLAETQYF,TRBV12-4,TRBJ2-5,1185
2892,3053,12,0.000014,TGTGCCAGCAGCCAAGAGACCCAGTACTTC,CASSQETQYF,TRBV3-1,TRBJ2-5,1185
3453,3756,10,0.000012,TGCGCCAGCAGCCAACGTGAAGAGACCCAGTACTTC,CASSQREETQYF,TRBV4-1,TRBJ2-5,1185
3639,3768,10,0.000012,TGTGCCAGCAGCTTCCAAGAGACCCAGTACTTC,CASSFQETQYF,TRBV7-9,TRBJ2-5,1185
3643,3785,10,0.000012,TGTGCCAGCAGCTTGGAGACCCAGTACTTC,CASSLETQYF,TRBV11-2,TRBJ2-5,1185
3650,3769,10,0.000012,TGTGCCAGCAGCTTGGGGGAGACCCAGTACTTC,CASSLGETQYF,TRBV5-6,TRBJ2-5,1185


In [ ]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [ ]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,1185,CASSLEETQYF,0.000017,TRBV5-1,TRBJ2-5,TRBV7-8*01,TRBJ2-5*01,MHCI,HomoSapiens,SARS-CoV-2,Nucleocapsid
1,1185,CASSLEETQYF,0.000017,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
2,1185,CASSLEETQYF,0.000017,TRBV5-1,TRBJ2-5,TRBV7-2*01,TRBJ2-5*01,MHCII,HomoSapiens,M.tuberculosis,CFP10
3,1185,CASSLETQYF,0.000017,TRBV11-2,TRBJ2-5,TRBV5-5*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,pp65
4,1185,CASSLGETQYF,0.000016,TRBV11-2,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
...,...,...,...,...,...,...,...,...,...,...,...
60,1185,CASSLGETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV11-2*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,pp65
61,1185,CASSLEETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV7-8*01,TRBJ2-5*01,MHCI,HomoSapiens,SARS-CoV-2,Nucleocapsid
62,1185,CASSLEETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
63,1185,CASSLEETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV7-2*01,TRBJ2-5*01,MHCII,HomoSapiens,M.tuberculosis,CFP10


# Разборки с V и J

In [ ]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
1,1185,CASSLEETQYF,0.000017,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
9,1185,CASSLGETQYF,0.000016,TRBV11-2,TRBJ2-5,TRBV11-2*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,pp65
11,1185,CASSQETQYF,0.000014,TRBV3-1,TRBJ2-5,TRBV3-1*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,IE1
37,1185,CASSLGETQYF,0.000007,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
40,1185,CASSLGETQYF,0.000007,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
55,1185,CASSLGETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
58,1185,CASSLGETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
62,1185,CASSLEETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
